<a href="https://colab.research.google.com/github/smartinternz02/SI-GuidedProject-610134-1699032511/blob/main/CrimeVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
#upload kaggle.json to access Kaggle Datasets
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
#download ucf-crime-dataset
!kaggle datasets download -d odins0n/ucf-crime-dataset

100% 11.0G/11.0G [01:16<00:00, 212MB/s]
100% 11.0G/11.0G [01:16<00:00, 154MB/s]


In [ ]:
!unzip ucf-crime-dataset.zip

In [75]:
#import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, AveragePooling2D, Dropout,MaxPooling2D, Conv2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input
from keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image

In [10]:
train_path = '/content/Train'
test_path = '/content/Test'

In [20]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [21]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [22]:
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1266345 images belonging to 14 classes.


In [23]:
test_set = test_datagen.flow_from_directory(test_path,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 111308 images belonging to 14 classes.


In [24]:
class_names = list(train_set.class_indices.keys())
class_names = np.array(class_names)
class_names

array(['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion',
       'Fighting', 'NormalVideos', 'RoadAccidents', 'Robbery', 'Shooting',
       'Shoplifting', 'Stealing', 'Vandalism'], dtype='<U13')

##Model Building

In [ ]:
#Using CNN and Transfer learning (pretrained model used - ResNet50)
model = Sequential()
resnet50 = ResNet50(include_top=False,
                   input_shape=(64,64,3),
                   pooling='avg',classes=14,
                   weights='imagenet')

for layer in resnet50.layers:
        layer.trainable=False

model.add(resnet50)
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation="relu"))
model.add(Dense(14, activation='softmax'))
model.summary()

In [46]:
#Compile model (other optimizers may result in better accuracy)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

##Training the Model

In [65]:
#Checkpoint used to save the best performances only
#EarlyStopping used to reduce training time
checkpoint = ModelCheckpoint(
    filepath='crime.h5',
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    verbose=1)

early_stopping_monitor = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    mode="auto",
    restore_best_weights=True,
    verbose=1,
)

In [ ]:
#training the model (taking a lotttt of time, but it works)
callbacks = [checkpoint,early_stopping_monitor]
epochs=10

#zyada hi time mile toh increase epochs for training, you might also want to increase steps_per_epoch idk..
model_history = model.fit(
  train_set,
  validation_data=test_set,
  epochs=epochs,
  steps_per_epoch = 100,
  callbacks=callbacks
)

##Testing the Model

In [67]:
from keras.models import load_model
model.load_weights('crime.h5')

In [ ]:
#fully trained model hopefully predicts better than it does rn :)
from keras.utils import load_img
img = image.load_img('/content/Test/RoadAccidents/RoadAccidents001_x264_0.png',target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
prediction = np.argmax(model.predict(x))
class_names[prediction]